### Importando o pandas, lendo os arquivos, juntando as duas tabelas e fazendo a limpeza dos dados

In [ ]:
import pandas as pd 

In [ ]:
#Transformei os arquivos para XLSX pois em csv estava dando um erro de encoding para UTF-8, e mesmo colocando 'enconding = utf-8', não deu certo, então, tive essa ideia de transformar para XLSX 
df_Details = pd.read_excel('desafio_details.xlsx')
df_Rating = pd.read_excel('desafio_ratings.xlsx')
#Df com as cidades colocadas a mão
df = pd.read_excel('df_clear (2).xlsx')

In [ ]:
df_Details.head()

In [ ]:
df_Rating.tail()

In [ ]:
df_Rating.drop(df_Rating[df_Rating['number_of_ratings'] == 0].index, inplace=True)

In [ ]:
df_RD = pd.concat([df_Rating, df_Details], axis=0)
df_RD.to_excel("df_RD.xlsx")
df_RD

In [ ]:
num_duplicados = df_RD["hotel_id"].duplicated().sum()
num_duplicados

In [ ]:
df_RD2 = df_RD.drop_duplicates(subset=["hotel_id"])

ids_para_excluir = ["10891689", "10846059", "10845958", "10037557", "9724595", "8726020", "8670985", "8262079", "7894257", "7869137"]

df_RD3 = df_RD2[~df_RD2["hotel_id"].isin(ids_para_excluir)]
df_RD3 = df_RD3.drop(df_RD3.columns[0], axis=1)
df_RD3

In [ ]:
nomes_para_excluir = ["excluir-CMR1101", "excluir - teste", "excluir -teste", "excluir-RAF004", "excluir - PRI", "Excluir", "Excluir", "Excluir - Suíte com vista piscina em resort de luxo ILC4202 - duplicado", "Excluir - Studio novo e decorado próx praia Santinho EDR203", "EXCLUIR Chalé do Papai Noel em Gramado RAR065", "excluir", "excluir- PCO1905", "excluir- ILC4112", "excluir-EAC403"]

df_clear = df_RD3[~df_RD3['hotel_name'].isin(nomes_para_excluir)]
df_clear = df_clear.drop('room_id', axis=1)
df_clear.tail()
#df_clear.to_excel("df_clear.xlsx")

In [ ]:
df_clear['city_name'].fillna('s/i', inplace=True)

In [ ]:
contagem_zeros = (df_clear['number_of_ratings'] == 0).sum()

print(f"Os anuncios que possui 'number_of_ratings' igual a zero é: {contagem_zeros}")

###### Nesses comentados, tentei completar os listings que não tinha url com a coluna hotel_name pois, achei que o nome que estava nessa coluna era o que tinha no link, mas foi um equívoco 

In [ ]:
#df_clear['hotel_name'] = df_clear['hotel_name'].str.replace(' ', '-') # :(

In [ ]:
#df_clear['url'] = df_clear.apply(
    #lambda row: f"https://www.booking.com/hotel/br/{row['hotel_name']}" if pd.isna(row['url']) else row['url'],
    ##)
# :(

#### 1. Ordene as cidades em ordem crescente de número de listings;

In [ ]:
listings = df["city_name"].value_counts()
listings

In [ ]:
cidades_ordenadas = listings.sort_values(ascending=True).to_frame().reset_index()
cidades_ordenadas

#### 2. Ordene as cidades em ordem decrescente de metros quadrados;


In [ ]:
total_m2 = df_clear.groupby("city_name")["room_surface_in_m2"].agg(sum)
cidades_ordenadas = total_m2.sort_values(ascending=False).to_frame().reset_index()
cidades_ordenadas

#### 3. Quais cidades têm mais avaliações?

In [ ]:
total_avaliacoes = df.groupby("city_name")['number_of_ratings'].sum()
cidades_mais_avaliacoes = total_avaliacoes.sort_values(ascending=False).to_frame().reset_index()
cidades_mais_avaliacoes

###### Aqui eu tentei fazer um webscraping para pegar as notas de avaliações dos usuários para ter mais dados para a análise, mas ainda assim não adiantou muito pois, tem muitos dados sem url do anúncio

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_guest_ratings(url):
    # Fazer uma solicitação HTTP para a página
    response = requests.get(url)

    # Verificar se a solicitação foi bem-sucedida (código de status 200)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Encontrar todos os elementos de avaliação dos hóspedes
        review_elements = soup.find_all('div', {'class': 'bui-spacer--larger'})

        # Extrair as notas de todas as avaliações
        guest_ratings = [element.text for element in review_elements]

        # Extrair as categorias de avaliação
        categories = [category.split()[0] for category in guest_ratings]

        # Criar um DataFrame com as notas divididas por categoria
        df = pd.DataFrame({'URL': [url] * len(guest_ratings)})
        
        for category, rating in zip(categories, guest_ratings):
            column_name = f"{category}_Rating"
            df[column_name] = [rating] * len(guest_ratings)

        return df
    else:
        print(f"Erro ao acessar a página {url}. Código de status: {response.status_code}")
        return None

urls = [
    'https://www.booking.com/hotel/br/apto-reformado-em-pousada-jurere-tradicional-psd103.html',
    'https://www.booking.com/hotel/br/asa-norte-brasilia-nor.html'
    'https://www.booking.com/hotel/br/metropolitan-sidney-mss.html'
    'https://www.booking.com/hotel/br/dna-smart-style-dna.html'
    'https://www.booking.com/hotel/br/maravilhoso-e-elegante-studio-para-casal-pgo412.html'
    'https://www.booking.com/hotel/br/apto-em-frente-ao-shopping-flamboyant-esp2906.html']


# Inicializar um DataFrame vazio
final_df = pd.DataFrame()

# Chamar a função para cada URL na lista e adicionar os resultados ao DataFrame
for url in urls:
    result_df = extract_guest_ratings(url)
    if result_df is not None:
        final_df = pd.concat([final_df, result_df], ignore_index=True)

# Exibir o DataFrame final
print(final_df)



#### 4. Quais cidades têm a maior média de avaliações? E a menor média?

In [ ]:
media_por_cidade = df.groupby('city_name')['Total'].mean().reset_index()

cidade_maior_media = media_por_cidade.loc[media_por_cidade['Total'].idxmax()]
cidade_menor_media = media_por_cidade.loc[media_por_cidade['Total'].idxmin()]

print(f'Cidade com a maior média: {cidade_maior_media["city_name"]} - Média: {cidade_maior_media["Total"]}')
print(f'Cidade com a menor média: {cidade_menor_media["city_name"]} - Média: {cidade_menor_media["Total"]}')


#### 6. Existem relações entre a nota do anúncio e os recursos disponíveis no imóvel? 


In [69]:

corr = df[['Total', 'Comodidades',	'Limpeza',	'Conforto',	'Custo-benefício',	'Localização']].corr()
print(corr)


                    Total  Comodidades   Limpeza  Conforto  Custo-benefício  \
Total            1.000000     0.852476  0.739320  0.775175         0.836138   
Comodidades      0.852476     1.000000  0.751714  0.737724         0.811097   
Limpeza          0.739320     0.751714  1.000000  0.678202         0.720722   
Conforto         0.775175     0.737724  0.678202  1.000000         0.773569   
Custo-benefício  0.836138     0.811097  0.720722  0.773569         1.000000   
Localização      0.555776     0.455359  0.512942  0.394349         0.476536   

                 Localização  
Total               0.555776  
Comodidades         0.455359  
Limpeza             0.512942  
Conforto            0.394349  
Custo-benefício     0.476536  
Localização         1.000000  
